In [1]:
import pandas as pd
import json
import ipywidgets as widgets
from IPython import display

NUMBER_OF_NODES = 10

lines = sc.textFile('file:///home/duan/github/customer_segmentation/data/journey_log/month-20160501.numbered')
lines_nonempty = lines.filter( lambda x: len(x) > 0 )

cust_df = pd.DataFrame.from_csv('/home/duan/github/customer_segmentation/data/cust_profile_df/cust_sentiment_gender_df.csv')
global_selected_cust_df = cust_df
cust_journey_df=pd.DataFrame.from_csv('/home/duan/github/customer_segmentation/data/cust_profile_df/cust_journey_df.csv')


In [2]:
cust_df

,newscore,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?,cust_id,state_cd,sentiment,cust_id_m,gender
0,2.0,128,415,382-4657,no,yes,25,265.1,110,45.07,...,10.0,3,2.70,1,False.,1,KS,7.0,NaN,F
1,2.0,107,415,371-7191,no,yes,26,161.6,123,27.47,...,13.7,3,3.70,1,False.,2,OH,5.0,2.0,M
2,2.0,75,415,330-6626,yes,no,0,166.7,113,28.34,...,10.1,3,2.73,3,False.,5,OK,5.0,5.0,M
3,2.0,121,510,355-9993,no,yes,24,218.2,88,37.09,...,7.5,7,2.03,3,False.,7,MA,6.0,7.0,M
4,2.0,74,415,344-9403,no,no,0,187.7,127,31.91,...,9.1,5,2.46,0,False.,12,RI,5.0,12.0,M
5,2.0,168,408,363-1107,no,no,0,128.8,96,21.90,...,11.2,2,3.02,1,False.,13,IA,4.0,NaN,F
6,2.0,95,510,394-8006,no,no,0,156.6,88,26.62,...,12.3,5,3.32,3,False.,14,MT,5.0,14.0,M
7,2.0,62,415,366-9238,no,no,0,120.7,70,20.52,...,13.1,6,3.54,4,False.,15,IA,4.0,15.0,M
8,2.0,161,415,351-7269,no,no,0,332.9,67,56.59,...,5.4,9,1.46,4,True.,16,NY,8.0,16.0,M
9,2.0,73,415,373-2782,no,no,0,224.4,90,38.15,...,13.0,2,3.51,1,False.,20,TX,6.0,20.0,M


In [3]:
from nvd3piewidget import (jouney_covered_check, 
                           collect_customer_from_journey, 
                           build_journey_with_customer_filter,
                           create_sankey_json
                          )
from nvd3piewidget import D3SankeyWidget,NVD3PieWidget,NVD3BarWidget


In [4]:
# Write the journey df to disk for later on computation.
def build_journey_all(lines_nonempty):
    wordcounts = build_journey_with_customer_filter(lines_nonempty, cust_list_rdd = None)
    from pyspark.sql import SQLContext, Row
    sqlContext = SQLContext(sc)
    # Load a text file and convert each line to a Row.  
    journey_rows = wordcounts.map(lambda p: Row(visitor_count=p[0], journey= p[1][0], visitor_list=p[1][1]))
    # Infer the schema, and register the DataFrame as a table.
    schemaPeople = sqlContext.createDataFrame(journey_rows)
    # schemaPeople.registerTempTable("journey_table")
    cust_journey_df = schemaPeople.toPandas()
    cust_journey_df.to_csv('/home/duan/github/customer_segmentation/data/cust_profile_df/cust_journey_df.csv.new.1')
    return cust_journey_df
# Only for first time, or schedule it every night.
# cust_journey_df = build_journey_all(lines_nonempty)

In [5]:
def selected_cust_sankey_json(lines_nonempty, selected_cust_df,NUMBER_OF_NODES=10, sc=None, rank_assending = False):
    if selected_cust_df is None:
        cust_list_rdd = None
    else:
        cust_list = [tuple(x) for x in selected_cust_df[['cust_id','cust_id']].values]
        cust_list_rdd = sc.parallelize(cust_list)
    # print('testing sankey' + str(rank_assending))
    new_journey_rdd = build_journey_with_customer_filter(lines_nonempty,cust_list_rdd, rank_assending = rank_assending) # ,new_sankey_links, new_sankey_nodes
    new_sankey_json = create_sankey_json(new_journey_rdd, NUMBER_OF_NODES) # , rank_assending = rank_assending, sc=sc
    return json.dumps(new_sankey_json)


In [6]:
def find_rare_journey_dict():
    journey_rdd = build_journey_with_customer_filter(lines_nonempty,None, rank_assending = True) # ,new_sankey_links, new_sankey_nodes
    journey_rdd.take(10)
    sankey_links = sc.parallelize(journey_rdd.take(10)).flatMap(flatmap_jouney).map(split_map_journey).reduceByKey(lambda x,y:x+y).map(lambda x:(x[1],x[0])).sortByKey(False)
    sankey_nodes = sc.parallelize(journey_rdd.take(10)).flatMap(flatmap_jouney_nodes).map(lambda x: (x, 1)).reduceByKey(lambda x,y:x+y).map(lambda x:(x[1],x[0])).sortByKey(False)
    return build_json_from_node_list_and_link(sankey_links,sankey_nodes, NUMBER_OF_NODES)



In [7]:
NUMBER_OF_NODES

10

In [8]:
NUMBER_OF_NODES

10

In [9]:
def update_bar_chart(global_selected_cust_df):
    hist1 = global_selected_cust_df['sentiment'].value_counts()
    hist1.sort_index(ascending=True,inplace=True)
    hist_dict=hist1.to_dict()
    bar_value = [{"y": int(hist_dict[k]), "x": str(int(k))}  for k in hist_dict.keys() ]
    data_barchart=[{"values": bar_value, 
                    "key": "Sentiment Historgram", 
                    "yAxis": "1"}]
    bar_chart.json_data = json.dumps(data_barchart)
def update_pie_chart(global_selected_cust_df):
    hist1 = global_selected_cust_df['gender'].value_counts()
    # hist1.sort_index(ascending=True,inplace=True)
    hist_dict=hist1.to_dict()
    gender_dict = {'F':'Female', 'M':'Male'}
    bar_value = [{"value": int(hist_dict[k]), "label": gender_dict[k]}  
                 for k in hist_dict.keys() ]
    data_piechart=[{"values": bar_value, 
                    "key": "Sentiment Historgram" }]
    pie_chart.json_data = json.dumps(data_piechart)

def update_sankey_chart(global_selected_cust_df, rank_assending = False):
    
    new_json_string = selected_cust_sankey_json(lines_nonempty, global_selected_cust_df[['cust_id']] ,NUMBER_OF_NODES, sc, rank_assending)
    sankey_journey_widget.node_link_json = new_json_string
# sankey_journey_widget.node_link_json = json_string
# update_sankey_chart(global_selected_cust_df[global_selected_cust_df['gender']  == 'M' ] )

def update_all_chart(global_selected_cust_df):
    update_bar_chart(global_selected_cust_df)
    update_pie_chart(global_selected_cust_df)
    update_sankey_chart(global_selected_cust_df)
    cust_count_string = '# of Customers: ' + str(global_selected_cust_df['cust_id'].count() )
    comment_field.value = cust_count_string


In [10]:
from IPython.display import HTML
HTML('''
<style>
#allchart {
   height: 350px;
   overflow: hidden;
}
#barchart {
    width: 420px;
    height: 350px;
    float: right;
}
#piechart {
    float: none; /* not needed, just for clarification */
    height: 350px;
    width: 420px;
    overflow: hidden;
</style>

<div id="allchart" >
    <div id="barchart"  >  &nbsp; &nbsp;  Sentiment Distribution
        <svg ></svg>
    </div>
    <div id="piechart"  > &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  Gender Distribution
        <svg ></svg>
    </div>
</div>

''')

In [11]:
json_string = selected_cust_sankey_json(lines_nonempty, None)
sankey_journey_widget = D3SankeyWidget(node_link_json = json_string, height = 300, width = 960 )
display.display(sankey_journey_widget)

In [12]:
cust_df

,newscore,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?,cust_id,state_cd,sentiment,cust_id_m,gender
0,2.0,128,415,382-4657,no,yes,25,265.1,110,45.07,...,10.0,3,2.70,1,False.,1,KS,7.0,NaN,F
1,2.0,107,415,371-7191,no,yes,26,161.6,123,27.47,...,13.7,3,3.70,1,False.,2,OH,5.0,2.0,M
2,2.0,75,415,330-6626,yes,no,0,166.7,113,28.34,...,10.1,3,2.73,3,False.,5,OK,5.0,5.0,M
3,2.0,121,510,355-9993,no,yes,24,218.2,88,37.09,...,7.5,7,2.03,3,False.,7,MA,6.0,7.0,M
4,2.0,74,415,344-9403,no,no,0,187.7,127,31.91,...,9.1,5,2.46,0,False.,12,RI,5.0,12.0,M
5,2.0,168,408,363-1107,no,no,0,128.8,96,21.90,...,11.2,2,3.02,1,False.,13,IA,4.0,NaN,F
6,2.0,95,510,394-8006,no,no,0,156.6,88,26.62,...,12.3,5,3.32,3,False.,14,MT,5.0,14.0,M
7,2.0,62,415,366-9238,no,no,0,120.7,70,20.52,...,13.1,6,3.54,4,False.,15,IA,4.0,15.0,M
8,2.0,161,415,351-7269,no,no,0,332.9,67,56.59,...,5.4,9,1.46,4,True.,16,NY,8.0,16.0,M
9,2.0,73,415,373-2782,no,no,0,224.4,90,38.15,...,13.0,2,3.51,1,False.,20,TX,6.0,20.0,M


In [13]:
data_barchart=[{'key': 'Sentiment Historgram',
  'values': [    {'x': '1', 'y':  75 },
   {'x': '2', 'y':  407 }, 
   {'x': '5', 'y':  204 }, 
   {'x': '7', 'y':  15 },
   {'x': '8', 'y':  1 }],'yAxis': '1'}]

bar_chart = NVD3BarWidget(json_data=json.dumps(data_barchart))
# print('         Sentiment Distribution')
display.display(bar_chart)
update_bar_chart(global_selected_cust_df)

In [14]:
data_piechart=[{'key': 'Sentiment Historgram', 
                'values': [{'label': 'F', 'value': 2007},{'label': 'M', 'value': 107}]}];

pie_chart = NVD3PieWidget(value=5, json_data=json.dumps(data_piechart))
# print('         Gender Distribution')
display.display(pie_chart)
update_pie_chart(global_selected_cust_df)

In [15]:
'''
global_selected_cust_df = cust_df[cust_df['gender']  == 'F' ]

update_bar_chart(global_selected_cust_df)
update_pie_chart(global_selected_cust_df)
update_sankey_chart(global_selected_cust_df)
'''

"\nglobal_selected_cust_df = cust_df[cust_df['gender']  == 'F' ]\n\nupdate_bar_chart(global_selected_cust_df)\nupdate_pie_chart(global_selected_cust_df)\nupdate_sankey_chart(global_selected_cust_df)\n"

In [16]:
# Buttons: Reset, Select


filter_by_journey_button = widgets.Button(description="Filter by Journey")
filter_by_journey_button.button_style = 'info'
filter_by_journey_button.margin = 5
filter_by_journey_button.height = 3

display.display(filter_by_journey_button)



def on_filter_by_journey_button_clicked(b):
    global global_selected_cust_df
    # print(d3sankeyExample.selected_json)
    if len(sankey_journey_widget.selected_json) < 3 :
        print('please select at least one node.')
        return
    ss_json = json.loads(sankey_journey_widget.selected_json)
    new_ss = {k: v for k, v in ss_json.items() if v == "y"}
    selected_journey=new_ss.keys()
    selected_journey_df=cust_journey_df[cust_journey_df.apply(lambda x: jouney_covered_check(selected_journey,x['journey']) , axis=1)]
    collecteddf = {'cust_id':  collect_customer_from_journey(selected_journey_df)}
    df_new = pd.DataFrame(collecteddf, columns = ['cust_id' ])
    df_new = df_new.apply(pd.to_numeric, errors='ignore').sort_values(by='cust_id')
    collected_df = pd.merge(df_new, cust_df, how='inner', left_on='cust_id', right_on='cust_id')
    global_selected_cust_df = collected_df
    
    update_all_chart(global_selected_cust_df) 

filter_by_journey_button.on_click(on_filter_by_journey_button_clicked)

In [17]:
# cust_journey_df[cust_journey_df['journey'] == 'phone_call:0,app:1,phone_call:2,web_chat:3,buy']

In [18]:
'''ss_json = json.loads(sankey_journey_widget.selected_json)
new_ss = {k: v for k, v in ss_json.items() if v == "y"}
selected_journey=new_ss.keys()
selected_journey_df=cust_journey_df[cust_journey_df.apply(lambda x: jouney_covered_check(selected_journey,x['journey']) , axis=1)]
collecteddf = {'cust_id':  collect_customer_from_journey(selected_journey_df)}

df_new = pd.DataFrame(collecteddf, columns = ['cust_id' ])
df_new = df_new.apply(pd.to_numeric, errors='ignore').sort_values(by='cust_id')
collected_df = pd.merge(df_new, cust_df, how='inner', left_on='cust_id', right_on='cust_id')
'''

'ss_json = json.loads(sankey_journey_widget.selected_json)\nnew_ss = {k: v for k, v in ss_json.items() if v == "y"}\nselected_journey=new_ss.keys()\nselected_journey_df=cust_journey_df[cust_journey_df.apply(lambda x: jouney_covered_check(selected_journey,x[\'journey\']) , axis=1)]\ncollecteddf = {\'cust_id\':  collect_customer_from_journey(selected_journey_df)}\n\ndf_new = pd.DataFrame(collecteddf, columns = [\'cust_id\' ])\ndf_new = df_new.apply(pd.to_numeric, errors=\'ignore\').sort_values(by=\'cust_id\')\ncollected_df = pd.merge(df_new, cust_df, how=\'inner\', left_on=\'cust_id\', right_on=\'cust_id\')\n'

In [19]:
# selected_journey
# del flatmap_jouney_nodes
from nvd3piewidget import (flatmap_jouney_nodes,flatmap_jouney,split_map_journey
                           )

In [20]:
# selected_journey_df

In [21]:
a=sorted(['store_visit:1', 'buy:99', 'store_visit:0', 'store_visit:2'])
a

['buy:99', 'store_visit:0', 'store_visit:1', 'store_visit:2']

In [22]:
'a' == 'b'

False

In [23]:
def jouney_covered_check(nodelist, journey):
    jnodes = journey.split(',')
    if(len(nodelist) != len(jnodes)):
        return False
    a = sorted(jnodes)
    #print(a)
    b = sorted(nodelist)
    #print(b)
    for i in range(len(jnodes)):
        if a[i] != b[i]:
            #print(i)
            #print(jnodes[i] + '=' + nodelist[i])
            return False
    return True
# jouney_covered_check(['store_visit:1', 'buy:99', 'store_visit:0', 'store_visit:2'],'store_visit:0,store_visit:1,buy')
# jouney_covered_check(['store_visit:1', 'buy', 'store_visit:0' ],'store_visit:0,store_visit:1,buy')

In [24]:
def jouney_containing_selected_node_check(selcted_nodelist, journey):
    jnodes = journey.split(',')
    for anode in jnodes:
        # print(anode)
        if anode in selcted_nodelist:
            return True
    return False
# jouney_containing_selected_node_check(['store_visit:0'],'store_visit:0,store_visit:1,buy')

In [25]:
# Buttons: Reset, Select


filter_by_nodea_button = widgets.Button(description="by Nodes (Any)")
filter_by_nodea_button.button_style = 'info'
filter_by_nodea_button.margin = 5
filter_by_nodea_button.height = 3

display.display(filter_by_nodea_button)



def on_filter_by_nodea_button_clicked(b):
    global global_selected_cust_df
    # print(d3sankeyExample.selected_json)
    if len(sankey_journey_widget.selected_json) < 3 :
        print('please select at least one node.')
        return
    ss_json = json.loads(sankey_journey_widget.selected_json)
    new_ss = {k: v for k, v in ss_json.items() if v == "y"}
    selected_journey=new_ss.keys()
    selected_journey_df=cust_journey_df[cust_journey_df.apply(lambda x: jouney_containing_selected_node_check(selected_journey,x['journey']) , axis=1)]
    collecteddf = {'cust_id':  collect_customer_from_journey(selected_journey_df)}
    df_new = pd.DataFrame(collecteddf, columns = ['cust_id' ])
    df_new = df_new.apply(pd.to_numeric, errors='ignore').sort_values(by='cust_id')
    collected_df = pd.merge(df_new, cust_df, how='inner', left_on='cust_id', right_on='cust_id')
    global_selected_cust_df = collected_df
    
    update_all_chart(global_selected_cust_df) 

filter_by_nodea_button.on_click(on_filter_by_nodea_button_clicked)

In [26]:
node_number_top_row = widgets.HBox()


node_number_field = widgets.Text()
node_number_field.height = 35
node_number_field.value='15'


set_nodes_number_button = widgets.Button(description="Number of Nodes")
set_nodes_number_button.button_style = 'info'
set_nodes_number_button.margin = 5
set_nodes_number_button.height = 3

node_number_top_row.children = ( node_number_field, set_nodes_number_button)
node_number_top_row.align = 'center'

display.display(node_number_top_row)

 


def on_set_nodes_number_button_clicked(b):
    global NUMBER_OF_NODES
    
    NUMBER_OF_NODES = int(node_number_field.value )
    
    global_selected_cust_df = cust_df
    update_all_chart(global_selected_cust_df) 


set_nodes_number_button.on_click(on_set_nodes_number_button_clicked)

In [27]:
least_button = widgets.Button(description="Rare Journey")
least_button.button_style = 'info'
least_button.margin = 5
least_button.height = 3
 
display.display(least_button)

 

# find_rare_journey_dict()
import json
def on_least_button_clicked(b):
    global cust_df
    global global_selected_cust_df

    global_selected_cust_df = cust_df
    new_json_string = json.dumps(find_rare_journey_dict())
    sankey_journey_widget.node_link_json = new_json_string
    

least_button.on_click(on_least_button_clicked)

In [28]:
new_journey_rdd = build_journey_with_customer_filter(lines_nonempty,None, rank_assending = True) # ,new_sankey_links, new_sankey_nodes

In [29]:
new_journey_rdd.take(10)

[(1, ('store_visit:0,app:1,app:2,phone_call:3,buy', '534')),
 (1, ('phone_call:0,app:1,phone_call:2,web_chat:3,buy', '1136')),
 (1, ('phone_call:0,web_chat:1,phone_call:2,not_buy', '1267')),
 (1, ('facebook:0,app:1,web_call:2,web_call:3,buy', '1638')),
 (1, ('store_visit:0,phone_call:1,phone_call:2,web_call:3,not_buy', '319')),
 (1, ('facebook:0,web_call:1,web_call:2,facebook:3,not_buy', '2771')),
 (1, ('phone_call:0,web_call:1,web_call:2,app:3,buy', '2137')),
 (1, ('store_visit:0,twitter:1,store_visit:2,twitter:3,not_buy', '1722')),
 (1, ('phone_call:0,app:1,not_buy', '2991')),
 (1, ('store_visit:0,twitter:1,app:2,buy', '1682'))]

In [30]:
'''
ss_json = json.loads(sankey_journey_widget.selected_json)
new_ss = {k: v for k, v in ss_json.items() if v == "y"}
selected_journey=new_ss.keys()
selected_journey

selected_journey_df=cust_journey_df[cust_journey_df.apply(lambda x: jouney_covered_check(selected_journey,x['journey']) , axis=1)]
collecteddf = {'cust_id':  collect_customer_from_journey(selected_journey_df)}

selected_journey
'''

'\nss_json = json.loads(sankey_journey_widget.selected_json)\nnew_ss = {k: v for k, v in ss_json.items() if v == "y"}\nselected_journey=new_ss.keys()\nselected_journey\n\nselected_journey_df=cust_journey_df[cust_journey_df.apply(lambda x: jouney_covered_check(selected_journey,x[\'journey\']) , axis=1)]\ncollecteddf = {\'cust_id\':  collect_customer_from_journey(selected_journey_df)}\n\nselected_journey\n'

In [31]:
jouney_covered_check(['store_visit:0'],'store_visit:0,store_visit:1,buy')

False

In [32]:
# 

In [33]:

filter_by_gender_button = widgets.Button(description="Filter by Gender")
filter_by_gender_button.button_style = 'info'
filter_by_gender_button.margin = 5
filter_by_gender_button.height = 3

display.display(filter_by_gender_button)



def on_filter_by_gender_button_clicked(b):
    global global_selected_cust_df
    if len(pie_chart.selected_json) < 3 :
        print('please select at least one gender.')
        return
    ss_json = json.loads(pie_chart.selected_json)
    new_ss = {k: v for k, v in ss_json.items() if v == "y"}
    reverse_gender_dict = {'Female':'F', 'Male':'M'}
    selected_gender=reverse_gender_dict [ list(new_ss.keys())[0] ]
    #print('gender=' + selected_gender)

    global_selected_cust_df = global_selected_cust_df[global_selected_cust_df['gender']== selected_gender] # == 'F'
    
    if(global_selected_cust_df['cust_id'].count() < 1):
        print('found no match, do not update charts')
        return
    # print(global_selected_cust_df.count()))
    update_all_chart(global_selected_cust_df) 

filter_by_gender_button.on_click(on_filter_by_gender_button_clicked)

In [34]:

filter_reset_button = widgets.Button(description="Reset Filters")
filter_reset_button.button_style = 'info'
filter_reset_button.margin = 5
filter_reset_button.height = 3

display.display(filter_reset_button)



def on_filter_reset_button_clicked(b):
    global global_selected_cust_df
    global cust_df
    
    global_selected_cust_df = cust_df
    update_all_chart(global_selected_cust_df) 

filter_reset_button.on_click(on_filter_reset_button_clicked)

In [35]:
'''
ss_json = json.loads(pie_chart.selected_json)
new_ss = {k: v for k, v in ss_json.items() if v == "y"}
selected_gender=list(new_ss.keys())[0]
global_selected_cust_df = global_selected_cust_df[global_selected_cust_df['gender'] 
                                                  == selected_gender] # == 'F'
update_bar_chart(global_selected_cust_df)
update_pie_chart(global_selected_cust_df)
update_sankey_chart(global_selected_cust_df)

'''

'\nss_json = json.loads(pie_chart.selected_json)\nnew_ss = {k: v for k, v in ss_json.items() if v == "y"}\nselected_gender=list(new_ss.keys())[0]\nglobal_selected_cust_df = global_selected_cust_df[global_selected_cust_df[\'gender\'] \n                                                  == selected_gender] # == \'F\'\nupdate_bar_chart(global_selected_cust_df)\nupdate_pie_chart(global_selected_cust_df)\nupdate_sankey_chart(global_selected_cust_df)\n\n'

In [36]:
grade_top_row = widgets.HBox()

hid_txt = widgets.HTML("Name:")
hid_txt.font = 'helvetica'
hid_txt.font_size = 30
hid_txt.margin = 5
algo_id_field = widgets.Text()
algo_id_field.height = 35
algo_id_field.value='Segment_1'

grade_txt = widgets.HTML("Type:")
grade_select = widgets.Dropdown(options=["Marketing", "Promotion", "Retension"])



grade_btn = widgets.Button(description="Create Segment")
grade_btn.margin = 5
grade_btn.button_style = 'info'


grade_top_row.children = (hid_txt, algo_id_field, grade_txt, grade_select, grade_btn)
grade_top_row.align = 'center'

comment_field = widgets.Textarea(value='')
comment_field.margin = 5
comment_field.width = 378
comment_field.cols = 55

grade_form = widgets.VBox()
grade_form.children = (grade_top_row, comment_field )


def on_create_button_clicked(b):
    # print(sankey_journey_widget.selected_json)
    if global_selected_cust_df['cust_id'].count() < 2 :
        print('Please select at least one customer.')
        return
    cust_list1 = global_selected_cust_df['cust_id'].tolist()
    cust_list = [str(int(x)) for x in cust_list1]
    # print(cust_list)
    new_seg = {'Seg_name': algo_id_field.value,
               'Seg_type': grade_select.value,
               'cust_ids': ','.join(cust_list)
              }
    display.clear_output(wait=True) 
    comment_field.value = json.dumps(new_seg)
    

display.display(grade_top_row)
grade_btn.on_click(on_create_button_clicked)


In [37]:
display.display(comment_field)
cust_count_string = '# of Customers: ' + str(global_selected_cust_df['cust_id'].count() )
comment_field.value = cust_count_string